In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

In [2]:
df = pd.read_stata('comp_dr.dta')
df

,gvkey,datadate,indfmt,consol,popsrc,datafmt,tic,conm,curcd,fyr,dlc,drc,drlt,lt,costat,LINKTYPE,LPERMNO,MthCalDt
0,001004,2000-05-31,INDL,C,D,STD,AIR,AAR CORP,USD,5.0,26.314,NaN,NaN,401.483,A,LU,54594.0,2000-05-31
1,001004,2001-05-31,INDL,C,D,STD,AIR,AAR CORP,USD,5.0,13.652,NaN,NaN,361.642,A,LU,54594.0,2001-05-31
2,001004,2002-05-31,INDL,C,D,STD,AIR,AAR CORP,USD,5.0,42.525,0.00,0.0,399.964,A,LU,54594.0,2002-05-31
3,001004,2003-05-31,INDL,C,D,STD,AIR,AAR CORP,USD,5.0,92.256,0.00,0.0,391.633,A,LU,54594.0,2003-05-30
4,001004,2004-05-31,INDL,C,D,STD,AIR,AAR CORP,USD,5.0,3.392,0.00,NaN,407.608,A,LU,54594.0,2004-05-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135043,349972,2022-12-31,INDL,C,D,STD,INDP,INDAPTUS THERAPEUTICS INC,USD,12.0,0.080,0.00,0.0,3.433,A,LC,15642.0,2022-12-30
135044,349972,2023-12-31,INDL,C,D,STD,INDP,INDAPTUS THERAPEUTICS INC,USD,12.0,0.102,0.00,0.0,2.847,A,LC,15642.0,2023-12-29
135045,351038,2022-12-31,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,12.0,0.000,0.00,0.0,7.051,A,LC,16161.0,2022-12-30
135046,351038,2023-12-31,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,12.0,0.000,0.00,0.0,6.505,A,LC,16161.0,2023-12-29


In [6]:
#current deferred revenue - drc
#total current liabilities - dlc
#long-term deferred revenue - drlt
#total long-term liabilities - lt
df = df[['gvkey', 'datadate', 'tic', 'drc', 'dlc', 'drlt', 'lt']]
df['dr'] = df['drc'] / df['lt']
df['datadate'] = pd.to_datetime(df['datadate'])

df['datadate'] = df['datadate'].dt.strftime('%Y-%m-%d')
df

,gvkey,datadate,tic,drc,dlc,drlt,lt,dr
0,001004,2000-05-31,AIR,NaN,26.314,NaN,401.483,NaN
1,001004,2001-05-31,AIR,NaN,13.652,NaN,361.642,NaN
2,001004,2002-05-31,AIR,0.00,42.525,0.0,399.964,0.000000
3,001004,2003-05-31,AIR,0.00,92.256,0.0,391.633,0.000000
4,001004,2004-05-31,AIR,0.00,3.392,NaN,407.608,0.000000
...,...,...,...,...,...,...,...,...
135043,349972,2022-12-31,INDP,0.00,0.080,0.0,3.433,0.000000
135044,349972,2023-12-31,INDP,0.00,0.102,0.0,2.847,0.000000
135045,351038,2022-12-31,QNRX,0.00,0.000,0.0,7.051,0.000000
135046,351038,2023-12-31,QNRX,0.00,0.000,0.0,6.505,0.000000
